In [1]:
# imports
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Emotions Dataset

In [2]:
emo_train = load_dataset("csv", data_files="./Swahili Emotion Data/emo_train.csv", encoding = "ISO-8859-1")
emo_valid = load_dataset("csv", data_files="./Swahili Emotion Data/emo_valid.csv", encoding = "ISO-8859-1")
emo_test = load_dataset("csv", data_files="./Swahili Emotion Data/emo_test.csv", encoding = "ISO-8859-1")

In [3]:
#  	0 - neutral
#	1 - joy (furaha)
#	2 - anger (hasira)
#	3 - sadness (huzuni)
#	4 - disgust (machukizo)
#	5 - suprise (mshangao)
#	6 - fear (woga)


classes = ['neutral','joy','anger','sadness','disgust','suprise','fear']
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

## Load Pre-Trained Model
### AfriBerta

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("castorini/afriberta_base")
model = AutoModelForTokenClassification.from_pretrained("castorini/afriberta_base", num_labels=len(classes),id2label=id2class, label2id=class2id,problem_type = "multi_label_classification")

tokenizer.model_max_length = 512 

c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of XLMRobertaForTokenClassificati

## Preprocess Data

In [5]:
# Preprocessing the Emotion Dataset using the Trainer

import ast

def process_label(data):
    lables = []
    for element in data['labels']:
        element = element.replace("[","")
        element = element.replace("]","")
        element = element.replace(" ","")
        element_list = element.split(",")
        label_list = [int(item) for item in element_list]
        lables.append(label_list)
    data['labels'] = lables
    return data

tokenised_trained_emotion = emo_train.map(process_label, batched=True)
tokenised_valid_emotion = emo_valid.map(process_label, batched=True)
tokenised_test_emotion = emo_test.map(process_label, batched=True)


def tokenize_function(data):
    all_labels = data['labels']
    labels = [0. for i in range(len(classes))]
    for label in all_labels:
        label_id = label
        labels[label_id] = 1.
    
    data = tokenizer(data["text"], padding = "max_length", truncation=True)
    data['labels'] = labels
    return data


tokenised_trained_emotion = tokenised_trained_emotion.map(tokenize_function)
tokenised_valid_emotion = tokenised_valid_emotion.map(tokenize_function)
tokenised_test_emotion = tokenised_test_emotion.map(tokenize_function)

tokenised_trained_emotion = tokenised_trained_emotion.rename_column("labels","label")
tokenised_valid_emotion = tokenised_valid_emotion.rename_column("labels","label")
tokenised_test_emotion = tokenised_test_emotion.rename_column("labels","label")

#print(tokenised_trained_emotion['train']['label'])

    
#small_train_dataset = tokenised_trained_emotion["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenised_valid_emotion["train"].shuffle(seed=42).select(range(1000))


In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [7]:
from transformers import TrainingArguments, Trainer

import evaluate


metric = evaluate.load("accuracy")
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

class CustomTrainer(Trainer):
   def compute_loss(self, model, inputs, return_outputs=False):
      outputs = model(
          input_ids=inputs['input_ids'],
          attention_mask=inputs['attention_mask'],
      )
      cls_logits = outputs['logits'][:, 0, :]
      loss = torch.nn.BCEWithLogitsLoss()(cls_logits.float(),
                                       inputs['labels'].float())
      return (loss, outputs) if return_outputs else loss
    

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0, :]
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    #predictions = np.argmax(logits, axis=1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))


#training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", learning_rate = 5e-5, adam_epsilon = 1e-8, lr_scheduler_type = "reduce_lr_on_plateau", lr_scheduler_kwargs = {'patience':5} )

training_args = TrainingArguments(
   #output_dir="my_awesome_model",
   #learning_rate=2e-5,
   learning_rate = 5e-5,
   adam_epsilon = 1e-8,
   lr_scheduler_type = "reduce_lr_on_plateau",
   lr_scheduler_kwargs = {'patience':5},
   #per_device_train_batch_size=4,
   #per_device_eval_batch_size=4,
   num_train_epochs=50,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_trained_emotion['train'],
    eval_dataset=tokenised_valid_emotion['train'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


trainer.train()

c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 500/60850 [02:20<4:42:21,  3.56it/s]

{'loss': 0.3287, 'grad_norm': 4.483614444732666, 'learning_rate': 5e-05, 'epoch': 0.41}


  2%|▏         | 1000/60850 [04:41<4:39:51,  3.56it/s]

{'loss': 0.2757, 'grad_norm': 1.8038887977600098, 'learning_rate': 5e-05, 'epoch': 0.82}


                                                      
  2%|▏         | 1217/60850 [05:56<4:01:41,  4.11it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


...

100%|██████████| 60850/60850 [4:56:37<00:00,  4.10it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
                                                       
100%|██████████| 60850/60850 [4:56:53<00:00,  4.10it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


{'eval_loss': 0.649949848651886, 'eval_accuracy': 0.885449939420641, 'eval_f1': 0.6248196248196248, 'eval_precision': 0.6316557257476295, 'eval_recall': 0.6181299072091363, 'eval_runtime': 14.0229, 'eval_samples_per_second': 92.491, 'eval_steps_per_second': 11.624, 'epoch': 50.0}


100%|██████████| 60850/60850 [4:56:55<00:00,  3.42it/s]

{'train_runtime': 17815.4471, 'train_samples_per_second': 27.313, 'train_steps_per_second': 3.416, 'train_loss': 0.01882769005310369, 'epoch': 50.0}


TrainOutput(global_step=60850, training_loss=0.01882769005310369, metrics={'train_runtime': 17815.4471, 'train_samples_per_second': 27.313, 'train_steps_per_second': 3.416, 'total_flos': 8.4771947234304e+16, 'train_loss': 0.01882769005310369, 'epoch': 50.0})

In [8]:
tokenizer.save_pretrained("./first_finetuning_model")
model.save_pretrained("./first_finetuning_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
